# Common Table Expression (CTE)

* A **common table expression** (CTE) allows you to create a temporary result set within a query.
* A CTE helps you enhance the readability of a complex query by breaking it down into smaller and more reusable parts.

**SYNTAX**
```postgresql
WITH cte_name (column1, column2, ...) AS (
    -- CTE query
    SELECT ...
)
-- Main query using the CTE
SELECT ...
FROM cte_name; 
```

> The column list is optional and is only necessary if you want to explicitly define the columns for the CTE.


**Basic PostgreSQL common table expression example**

```postgresql
WITH action_films AS (
  SELECT
    f.title,
    f.length
  FROM film f
  INNER JOIN film_category fc USING (film_id)
  INNER JOIN category c USING(category_id)
  WHERE c.name = 'Action'
)
    
SELECT * FROM action_films; 
```

**Join a CTE with a table**

```postgresql
WITH cte_rental AS (
  SELECT
    staff_id,
    COUNT(rental_id) rental_count
  FROM rental
  GROUP BY staff_id
)

    
SELECT
  s.staff_id,
  first_name,
  last_name,
  rental_count
FROM staff s
INNER JOIN cte_rental USING (staff_id);
```

**Multiple CTEs**

```postgresql
WITH film_stats AS (
    -- CTE 1: Calculate film statistics
    SELECT
        AVG(rental_rate) AS avg_rental_rate,
        MAX(length) AS max_length,
        MIN(length) AS min_length
    FROM film
),
customer_stats AS (
    -- CTE 2: Calculate customer statistics
    SELECT
        COUNT(DISTINCT customer_id) AS total_customers,
        SUM(amount) AS total_payments
    FROM payment
)

    
-- Main query using the CTEs
SELECT
    ROUND((SELECT avg_rental_rate FROM film_stats), 2) AS avg_film_rental_rate,
    (SELECT max_length FROM film_stats) AS max_film_length,
    (SELECT min_length FROM film_stats) AS min_film_length,
    (SELECT total_customers FROM customer_stats) AS total_customers,
    (SELECT total_payments FROM customer_stats) AS total_payments;
```

**PostgreSQL CTE advantages**

The following are some advantages of using common table expressions or CTEs:
* **Improve the readability of complex queries**. You use CTEs to organize complex queries in a more organized and readable manner.
* Ability to create **recursive queries**, which are queries that reference themselves. The recursive queries come in handy when you want to query hierarchical data such as organization charts.
* Use in conjunction with window functions. You can use CTEs in conjunction with window functions to create an initial result set and use another select statement to further process this result set.

# Recursive Query

* In PostgreSQL, a **common table expression** (CTE) is a named temporary result set within a query.
* A **recursive CTE** allows you to perform recursion within a query using the `WITH RECURSIVE` syntax.
* A **recursive CTE** is often referred to as a recursive query.

**SYNTAX**
```postgresql

WITH RECURSIVE cte_name (column1, column2, ...) AS (
    -- anchor member
    SELECT column_list FROM table1 WHERE condition

    UNION [ALL]

    -- recursive term
    SELECT column_list FROM cte_name WHERE recursive_condition
)
    
SELECT * FROM cte_name;

```

**SAMPLE DATA**
```postgresql
INSERT INTO employees (employee_id, full_name, manager_id)
VALUES
  (1, 'Michael North', NULL),
  (2, 'Megan Berry', 1),
  (3, 'Sarah Berry', 1),
  (4, 'Zoe Black', 1),
  (5, 'Tim James', 1),
  (6, 'Bella Tucker', 2),
  (7, 'Ryan Metcalfe', 2),
  (8, 'Max Mills', 2),
  (9, 'Benjamin Glover', 2),
  (10, 'Carolyn Henderson', 3),
  (11, 'Nicola Kelly', 3),
  (12, 'Alexandra Climo', 3),
  (13, 'Dominic King', 3),
  (14, 'Leonard Gray', 4),
  (15, 'Eric Rampling', 4),
  (16, 'Piers Paige', 7),
  (17, 'Ryan Henderson', 7),
  (18, 'Frank Tucker', 8),
  (19, 'Nathan Ferguson', 8),
  (20, 'Kevin Rampling', 8); 
```

**EXAMPLE**:
```postgresql
WITH RECURSIVE subordinates AS (
  
  SELECT
    employee_id,
    manager_id,
    full_name
  FROM
    employees
  WHERE
    employee_id = 2
  
  UNION
    
  SELECT
    e.employee_id,
    e.manager_id,
    e.full_name
  FROM
    employees e
    INNER JOIN subordinates s ON s.employee_id = e.manager_id
)

    
SELECT * FROM subordinates;

```

**OUTPUT**
```postgresql
employee_id | manager_id |    full_name
-------------+------------+-----------------
           2 |          1 | Megan Berry
           6 |          2 | Bella Tucker
           7 |          2 | Ryan Metcalfe
           8 |          2 | Max Mills
           9 |          2 | Benjamin Glover
          16 |          7 | Piers Paige
          17 |          7 | Ryan Henderson
          18 |          8 | Frank Tucker
          19 |          8 | Nathan Ferguson
          20 |          8 | Kevin Rampling
(10 rows)
```